# Import Subhalo Properties from Galacticus Output

This notebook demonstrates importing subhalo properties from galacticus and incorporating into a realization

In [5]:
from pyHalo.Halos.galacticus_util.galacticus_util import GalacticusFileReader
from pyHalo.preset_models import DMFromGalacticus

galacticusfile = GalacticusFileReader.read_file("data/TNFW_example.hdf5")

In [6]:
realization = DMFromGalacticus(0.5,2,galacticusfile,{"cone_opening_angle_arcsec":10},(1E9,1E10),False,None,None,2)